# Configuração 


In [1]:
#!virtualenv --python=python3.8 venv #rodar apenas na primeira vez que rodar o script na maquina
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install seqeval
!pip install scikit-learn==0.24
!pip install sklearn==0.0
!pip install sklearn-crfsuite==0.3.6
!pip install python-crfsuite==0.9.7
!pip install tqdm
!pip install pytorch-crf==0.7.2
#!pip install torch==1.8.1
#!pip install torch==1.11.0
!pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
#!virtualenv --python=python3.8 venv
#source venv/bin/activate

created virtual environment CPython3.8.10.final.0-64 in 75ms
  creator CPython3Posix(dest=/home/lucelia_vieira/Experimentos/bilstm/venv, clear=False, global=False)
  seeder FromAppData(download=False, pip=latest, setuptools=latest, wheel=latest, pkg_resources=latest, via=copy, app_data_dir=/home/lucelia_vieira/.local/share/virtualenv/seed-app-data/v1.0.1.debian.1)
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator


In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
#from TorchCRF import CRF
from torchcrf import CRF
from tqdm import tqdm
import numpy as np
from sklearn import metrics
from sklearn_crfsuite import metrics

/home/lucelia_vieira/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#configuraçao para que as variáveis não mudem quando executadas nas mesmas condições pelo modelo
import random
import numpy as np
import torch
seed = 3
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
!export CUBLAS_WORKSPACE_CONFIG=:16:8

# Dados de notícias no padrão CoNLL

In [4]:
train_path = '/home/lucelia_vieira/Experimentos/Data/CoNLL/conll_train.txt'
sentences = open(train_path, 'r').readlines()

train_path = '/home/lucelia_vieira/Experimentos/Data/CoNLL/conll_valid.txt'
sentences = open(train_path, 'r').readlines()

train_path = '/home/lucelia_vieira/Experimentos/Data/CoNLL/conll_test.txt'
sentences = open(train_path, 'r').readlines()



# Pré-processamento dos dados

* Passo 1: Separar as palavras e as classes
* Passo 2: Criar dicionarios de palavras e classes (utilizados para treinar o modelo neural)
* Passo 3: Substituir as palavras pelo seu indice no dicionario criado
* Passo 4: Adicionar tokens de inicio e final de sentenca
* Passo 5: Criar os mini-batches de treinamento

### Passo 1: Separando as palavras e as classes 

Separando as palavras e as classes do CoNLL

In [52]:
def preprocess(ner_set):
    sentences = []
    tags = []

    temp_sentence = []
    temp_tag = []
    for line in ner_set:
        try:
            word, _, _, tag = line.split()
            temp_sentence.append(word)
            temp_tag.append(tag)
        except:
            sentences.append(temp_sentence)
            tags.append(temp_tag)
            temp_sentence = []
            temp_tag = []

    if temp_sentence:
        sentences.append(temp_sentence)
        tags.append(temp_tag)
    return sentences, tags

In [107]:
dataset = 'conll'

train_x, train_y = preprocess(open('/home/lucelia_vieira/Experimentos/Data/CoNLL/conll_train.txt', 'r'))
valid_x, valid_y = preprocess(open('/home/lucelia_vieira/Experimentos/Data/CoNLL/conll_valid.txt', 'r'))
test_x, test_y = preprocess(open('/home/lucelia_vieira/Experimentos/Data/CoNLL/conll_valid.txt', 'r'))

len(train_x), len(valid_x), len(test_x)

(14989, 3466, 3466)

In [90]:
print(' '.join(train_x[10]))
print()
print(' '.join(train_y[10]))

He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .

O O O O O O O B-ORG O O B-PER I-PER O O O O O O O O O O O O O O O O O O O O O O O O O O O O


Separando as palavras e as classes do Ontonotes

In [91]:
def preprocessO(ner_set):
    sentences = []
    tags = []

    temp_sentence = []
    temp_tag = []
    for line in ner_set:
        try:
            word, tag = line.split()
            temp_sentence.append(word)
            temp_tag.append(tag)
        except:
            sentences.append(temp_sentence)
            tags.append(temp_tag)
            temp_sentence = []
            temp_tag = []

    if temp_sentence:
        sentences.append(temp_sentence)
        tags.append(temp_tag)
    return sentences, tags

In [92]:
valid_x_o, valid_y_o = preprocessO(open('/home/lucelia_vieira/Experimentos/Data/Ontonotes/ner_train_label_teste.txt', 'r'))
len(valid_x_o)

28

In [93]:
print(' '.join(valid_x_o[10]))
print()
print(' '.join(valid_y_o[10]))

The man showed Elisha the place where the ax head fell .

O O O O O O O O O O O O


### Passo 2: Criando os dicionários de palavras e classes (utilizando o conjunto de treinamento)

Incluindo os tokens especiais \<UNK\>, \<BOS\>, \<EOS\> de palavras e \<PAD\> para classes

In [108]:
from collections import OrderedDict

def word_dict(sentences):
    word2idx = OrderedDict({'<UNK>': 0, '<PAD>': 1, '<BOS>': 2, '<EOS>': 3})
    for sentence in sentences:
        for word in sentence:
            if word not in word2idx:
                word2idx[word] = len(word2idx)
    return word2idx

def tag_dict(tag_sentences):
    tag2idx = OrderedDict({'<PAD>': 0})
    for tags in tag_sentences:
        for tag in tags:
            if tag not in tag2idx:
                tag2idx[tag] = len(tag2idx)
    return tag2idx

word2idx = word_dict(train_x)
tag2idx  = tag_dict(train_y)


In [95]:
tag2idx

OrderedDict([('<PAD>', 0),
             ('O', 1),
             ('B-ORG', 2),
             ('B-MISC', 3),
             ('B-PER', 4),
             ('I-PER', 5),
             ('B-LOC', 6),
             ('I-ORG', 7),
             ('I-MISC', 8),
             ('I-LOC', 9)])

In [109]:
for idx, word in enumerate(word2idx):
    print(word, word2idx[word])
    if idx >= 20:
        break

<UNK> 0
<PAD> 1
<BOS> 2
<EOS> 3
-DOCSTART- 4
EU 5
rejects 6
German 7
call 8
to 9
boycott 10
British 11
lamb 12
. 13
Peter 14
Blackburn 15
BRUSSELS 16
1996-08-22 17
The 18
European 19
Commission 20


In [110]:
print("Numero de classes por dataset")
idx2tag = {idx: tag for tag, idx in tag2idx.items()}
#print(idx2tag)
label = list(idx2tag.values())
label.remove('<PAD>')
label.remove('O')
print(label)

count_org = 0
count_misc = 0
count_per = 0
count_loc = 0

for i in label:
  #for j in train_y:
  for j in valid_y_o:
    if i in j:
        if i in ('B-ORG','I-ORG'):
          count_org +=1
        if i in ('B-MISC', 'I-MISC'):
          count_misc +=1
        if i in ('B-PER', 'I-PER'):
              count_per +=1
        if i in ('B-LOC', 'I-LOC'):
              count_loc +=1
print("Conjunto de treinamento:",('B-ORG', count_org))
print("Conjunto de treinamento:",('B-MISC', count_misc))
print("Conjunto de treinamento:",('B-PER', count_per))
print("Conjunto de treinamento:",('B-LOC', count_loc))

print("Conjunto de treinamento:",('Total de Tokens', count_org+count_misc+count_per+count_loc))


Numero de classes por dataset
['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']
Conjunto de treinamento: ('B-ORG', 10)
Conjunto de treinamento: ('B-MISC', 17)
Conjunto de treinamento: ('B-PER', 6)
Conjunto de treinamento: ('B-LOC', 6)
Conjunto de treinamento: ('Total de Tokens', 39)


### Passo 3: Substituir as palavras e classes pelos seus indices nos dicionarios criados

In [111]:
def numericalize(sentences, word2idx, tag_sentences, tag2idx):
    numericalized_sentences = [[word2idx['<UNK>'] if word not in word2idx else word2idx[word] for word in sentence] for sentence in sentences]
    numericalized_tags = [[tag2idx[tag] for tag in tags] for tags in tag_sentences]
    return numericalized_sentences, numericalized_tags

train_x, train_y = numericalize(train_x, word2idx, train_y, tag2idx)
valid_x, valid_y = numericalize(valid_x, word2idx, valid_y, tag2idx)
test_x, test_y = numericalize(test_x, word2idx, test_y, tag2idx)



In [104]:
train_x[30]

[335,
 336,
 83,
 337,
 338,
 90,
 339,
 166,
 340,
 341,
 342,
 343,
 97,
 266,
 9,
 344,
 141,
 345,
 75,
 194,
 161,
 346,
 347,
 348,
 349,
 74,
 350,
 291,
 134,
 271,
 351,
 136,
 13]

### Passo 4: Adicionar tokens de inicio e final de sentenca

In [112]:
import itertools
import torch
def add_special_tokens(sentences, word2idx, tag_sentences, tag2idx):
    formatted_sentences = [torch.LongTensor([word for word in itertools.chain([word2idx['<BOS>']], sentence, [word2idx['<EOS>']])]) for sentence in sentences]
    formatted_tags = [torch.LongTensor([tag for tag in itertools.chain([tag2idx['O']], tags, [tag2idx['O']])]) for tags in tag_sentences]
    return formatted_sentences, formatted_tags

train_x, train_y = add_special_tokens(train_x, word2idx, train_y, tag2idx)
valid_x, valid_y = add_special_tokens(valid_x, word2idx, valid_y, tag2idx)
test_x, test_y   = add_special_tokens(test_x, word2idx, test_y, tag2idx)

train_x[31]


tensor([  2,  18, 352, 353, 166,  83, 345, 354, 240, 355, 356, 357,  13,   3])

### Passo 5: Criar os mini-batches de treinamento

In [113]:
import numpy as np

# Ordenando sentencas por tamanho (antes de criar os batches)
ordered_idx = np.argsort([len(train_x[i]) for i in range(len(train_x))])
train_x = [train_x[idx] for idx in ordered_idx]
train_y = [train_y[idx] for idx in ordered_idx]

In [114]:
from torch.nn.utils.rnn import pad_sequence

def create_batches(x, y, batch_size, pad_token, pad_class):
    batch_x = []
    batch_y = []
    mask = []

    # Separando os batches pelo tamanho de batch_size
    i = 0
    while i < len(x):
        batch_x.append(x[i:min(len(x), i+batch_size)])
        batch_y.append(y[i:min(len(y), i+batch_size)])
        i += batch_size
    
    # Realizando padding dos batches e criando mask para identificar padding durante o treinamento
    for i in range(len(batch_x)):
        batch_x[i] = pad_sequence(batch_x[i], batch_first = True, padding_value = pad_token)
        batch_y[i] = pad_sequence(batch_y[i], batch_first = True, padding_value = pad_class)
        mask.append(batch_x[i] != pad_token)

    return batch_x, batch_y, mask

train_x, train_y, mask = create_batches(train_x, train_y, batch_size=32, pad_token=word2idx['<PAD>'], pad_class=tag2idx['<PAD>'])
#valid_x, _, valid_mask = create_batches(valid_x, valid_y, batch_size=32, pad_token=word2idx['<PAD>'], pad_class=tag2idx['<PAD>'])
valid_x, _, valid_mask = create_batches(valid_x, valid_y, batch_size=32, pad_token=word2idx['<PAD>'], pad_class=tag2idx['<PAD>'])
test_x, _, test_mask   = create_batches(test_x,  test_y,  batch_size=32,  pad_token=word2idx['<PAD>'], pad_class=tag2idx['<PAD>'])

In [115]:
#train_x[0].shape, train_y[0].shape, mask[0].shape
train_x[0].shape, train_y[0].shape, mask[0].shape, test_x[0].shape, test_y[0].shape

(torch.Size([32, 3]),
 torch.Size([32, 3]),
 torch.Size([32, 3]),
 torch.Size([32, 43]),
 torch.Size([3]))

In [116]:
#train_x[0][0, :mask[0][0].sum()]
train_x[0][0], mask[0][0]

(tensor([2, 3, 1]), tensor([ True,  True, False]))

# Criando o modelo neural biLSTM-CRF

O modelo consiste em:



1.   Camada de embeddings (transforma indices das palavras em vetores numericos)
2.   Camada biLSTM para encode da informação (-> <-)
3.   Camada linear para reducao da dimensao do vetor de saida da camada biLSTM
3.   Camada CRF para classificação de cada token de entrada



In [117]:
from torchcrf import CRF

class bilstm_crf(torch.nn.Module):
  
    def __init__(self, word2idx, tag2idx):
        super(bilstm_crf, self).__init__()
        # Camada de embeddings
        self.embedding = torch.nn.Embedding(num_embeddings=len(word2idx), embedding_dim = 50, padding_idx = word2idx['<PAD>'])
        # Camada biLSTM
        self.bilstm = torch.nn.LSTM(input_size=50, hidden_size = 200, num_layers = 2, batch_first = True, bidirectional = True, dropout = 0.25)
        # Camada linear
        self.linear = torch.nn.Linear(400, len(tag2idx))
        # Camada CRF
        self.crf = CRF(num_tags = len(tag2idx), batch_first = True)

    def forward(self, x, y, mask):
        x = self.embedding(x)
        x, _ = self.bilstm(x)
        x = self.linear(x)
        loss = self.crf(x, y, mask=mask)
        return loss

    def decode(self, x, mask):
        x = self.embedding(x)
        x, _ = self.bilstm(x)
        x = self.linear(x)
        prediction = self.crf.decode(x, mask=mask)
        return prediction

In [145]:
len(train_x[0])

32

In [154]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = bilstm_crf(word2idx = word2idx, tag2idx = tag2idx)
model = model.to(device)
loss = model(train_x[0].to(device), train_y[0].to(device), mask[0].to(device))
#prediction = model.decode(valid_y[0].to(device), valid_mask[0].to(device))
prediction = model.decode(valid_x[0].to(device), valid_mask[0].to(device))

loss

cuda


tensor(-217.1929, device='cuda:0', grad_fn=<SumBackward0>)

In [155]:
prediction

[[4, 9, 3],
 [4, 9, 3, 4, 9, 3, 4, 9, 8, 3, 4, 9, 3],
 [4, 9, 3, 5],
 [4,
  9,
  8,
  3,
  4,
  9,
  3,
  4,
  9,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  1,
  9,
  1,
  9,
  1,
  9,
  3,
  4,
  9,
  3],
 [4,
  9,
  1,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  1,
  9,
  1,
  9,
  1,
  9,
  1,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  5],
 [4,
  9,
  1,
  9,
  8,
  3,
  4,
  9,
  3,
  4,
  9,
  3,
  4,
  9,
  8,
  3,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  1,
  9,
  1,
  9,
  1,
  9,
  8,
  3,
  5,
  2,
  2,
  2,
  2,
  9,
  3,
  5],
 [1,
  9,
  8,
  3,
  4,
  9,
  1,
  9,
  1,
  9,
  8,
  3,
  4,
  9,
  3,
  4,
  9,
  1,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3],
 [4,
  9,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  1,
  9,
  3,
  4,
  9,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  4,
  9,
  8,
  3,
  5],
 [4,
  9,
  1,
  9,
  1,
  9,
  1,
  9,
  8,
  3,
  4,
 

In [120]:
# To save trained pytorch model
torch.save(model.state_dict(), "./modelo/modelo_inicial")

# # To load trained pytorch model
# loaded_model = LSTM_CRF(embedding_dim=100, word2idx_dict=dic, num_tags=3, hidden_dim=128)
# new_model.load_state_dict(torch.load("seg_model"))

In [121]:
#Para rodar o pyTorch no cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda


In [122]:
import torch
import sys
print('Python Version', sys.version)
print('Torch Version', torch.__version__)
print('Cuda available', torch.cuda.is_available())

print('Cuda Version',torch.version.cuda)
print(torch.backends.cudnn.version())
#print(torch.ls cuda.get_device_name(0))

print('D', torch.backends.cudnn.enabled)
device = torch.device('cuda')
print('E', torch.cuda.get_device_properties(device))
print('F', torch.tensor([1.0, 2.0]).cuda())

Python Version 3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]
Torch Version 1.11.0+cu113
Cuda available True
Cuda Version 11.3
8200
D True
E _CudaDeviceProperties(name='NVIDIA GeForce RTX 3060', major=8, minor=6, total_memory=12053MB, multi_processor_count=28)
F tensor([1., 2.], device='cuda:0')


Separando as palavras e as classes do CoNLL

# Código para avaliação do modelo

Métrica span-based f1-score (f1-score a nível de entidades)

$f_1 = 2\frac{precision\times recall}{precision + recall}$

$precision = \frac{TP}{TP + FP}$

$recall = \frac{TP}{TP + FN}$

In [123]:
from seqeval.metrics import f1_score
from seqeval.scheme import IOB1

In [161]:
idx2tag = {idx: tag for tag, idx in tag2idx.items()}

def IOBify(tags_sequence, idx2tag):
    if isinstance(tags_sequence[0], list):
        iob_y = [[idx2tag[tag] for tag in tags] for tags in tags_sequence]
    else:
        iob_y = [[idx2tag[tag.item()] for tag in tags] for tags in tags_sequence]
    return iob_y

y_true = IOBify(valid_y, idx2tag)
#y_true = IOBify(valid_x[0].tolist(), idx2tag)

y_pred = IOBify(prediction, idx2tag)

KeyError: 2177

In [160]:
len(valid_x[0]), len(y_pred)


(32, 32)

In [138]:
prediction

[[3, 9, 3],
 [3, 2, 5, 2, 5, 2, 3, 9, 0, 6, 8, 0, 5],
 [1, 6, 0, 5],
 [1,
  0,
  6,
  8,
  0,
  6,
  0,
  6,
  8,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  9,
  3,
  9,
  3,
  9,
  3,
  9,
  0,
  6,
  6,
  0,
  6,
  8,
  0,
  6,
  8,
  0,
  6,
  0,
  6,
  9],
 [1,
  0,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  6,
  6,
  0,
  6,
  0,
  4,
  1,
  0,
  4,
  1,
  6,
  0,
  6,
  6,
  6,
  7,
  3],
 [3,
  9,
  3,
  9,
  0,
  6,
  0,
  6,
  8,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  8,
  0,
  4,
  0,
  4,
  0,
  6,
  0,
  6,
  0,
  4,
  1,
  0,
  6,
  9],
 [1,
  6,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  8,
  0,
  5],
 [1,
  6,
  6,
  0,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  8,
  0,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  6,
  7,
  3,
  9,
  3,
  9],
 [1,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  0,
 

In [125]:
prediction

[[3, 9, 3],
 [3, 2, 5, 2, 5, 2, 3, 9, 0, 6, 8, 0, 5],
 [1, 6, 0, 5],
 [1,
  0,
  6,
  8,
  0,
  6,
  0,
  6,
  8,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  9,
  3,
  9,
  3,
  9,
  3,
  9,
  0,
  6,
  6,
  0,
  6,
  8,
  0,
  6,
  8,
  0,
  6,
  0,
  6,
  9],
 [1,
  0,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  6,
  6,
  0,
  6,
  0,
  4,
  1,
  0,
  4,
  1,
  6,
  0,
  6,
  6,
  6,
  7,
  3],
 [3,
  9,
  3,
  9,
  0,
  6,
  0,
  6,
  8,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  8,
  0,
  4,
  0,
  4,
  0,
  6,
  0,
  6,
  0,
  4,
  1,
  0,
  6,
  9],
 [1,
  6,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  8,
  0,
  5],
 [1,
  6,
  6,
  0,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  8,
  0,
  6,
  6,
  6,
  6,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  6,
  6,
  7,
  3,
  9,
  3,
  9],
 [1,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  0,
  6,
  0,
 

In [126]:
#Teste Lucelia
y_pred


[['B-MISC', 'I-LOC', 'B-MISC'],
 ['B-MISC',
  'B-ORG',
  'I-PER',
  'B-ORG',
  'I-PER',
  'B-ORG',
  'B-MISC',
  'I-LOC',
  '<PAD>',
  'B-LOC',
  'I-MISC',
  '<PAD>',
  'I-PER'],
 ['O', 'B-LOC', '<PAD>', 'I-PER'],
 ['O',
  '<PAD>',
  'B-LOC',
  'I-MISC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  'I-MISC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  'B-LOC',
  'I-LOC',
  'B-MISC',
  'I-LOC',
  'B-MISC',
  'I-LOC',
  'B-MISC',
  'I-LOC',
  '<PAD>',
  'B-LOC',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  'I-MISC',
  '<PAD>',
  'B-LOC',
  'I-MISC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  'I-LOC'],
 ['O',
  '<PAD>',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  'B-LOC',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  '<PAD>',
  'B-PER',
  'O',
  '<PAD>',
  'B-PER',
  'O',
  'B-LOC',
  '<PAD>',
  'B-LOC',
  

3466

In [127]:
len(y_true[0])

3

In [128]:
len(y_pred), len(y_true[0])

(32, 3)

In [129]:
f1_score(y_true, y_pred)

ValueError: Found input variables with inconsistent numbers of samples:
[3, 13, 4, 37, 39, 40, 26, 32, 36, 41, 43, 27, 36, 33, 30, 24, 11, 3, 9, 4, 16, 14, 15, 9, 32, 20, 37, 27, 27, 29, 28, 3, 8, 4, 8, 11, 15, 6, 10, 13, 10, 11, 12, 4, 5, 3, 7, 8, 14, 6, 6, 6, 11, 4, 12, 4, 14, 12, 4, 13, 3, 13, 14, 10, 11, 14, 10, 3, 13, 8, 6, 10, 10, 10, 3, 10, 13, 10, 14, 3, 10, 4, 19, 42, 22, 36, 28, 30, 22, 30, 20, 16, 3, 8, 4, 6, 7, 15, 4, 3, 11, 4, 19, 4, 15, 9, 5, 3, 12, 4, 40, 31, 22, 19, 31, 3, 14, 5, 28, 25, 39, 9, 9, 6, 12, 41, 18, 30, 33, 21, 3, 10, 5, 34, 17, 21, 20, 3, 11, 4, 7, 7, 10, 10, 12, 17, 12, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 11, 4, 6, 9, 6, 6, 6, 6, 6, 6, 15, 11, 16, 7, 8, 9, 8, 9, 8, 9, 8, 9, 3, 12, 5, 23, 8, 16, 8, 22, 16, 18, 19, 18, 8, 22, 18, 8, 17, 18, 19, 47, 39, 44, 22, 3, 9, 13, 9, 15, 3, 13, 10, 9, 15, 4, 3, 45, 3, 12, 5, 5, 13, 11, 4, 4, 6, 8, 7, 8, 8, 8, 4, 7, 7, 8, 7, 8, 4, 7, 7, 8, 8, 7, 6, 5, 5, 5, 6, 5, 5, 4, 4, 6, 7, 7, 7, 8, 7, 4, 7, 9, 7, 7, 7, 4, 8, 8, 7, 8, 7,

# Código para treinamento do modelo

In [42]:
lrate = 0.01
optim = torch.optim.SGD(model.parameters(), lr=lrate, momentum=0.9)

In [43]:
f1_history = []
mean_loss_history = []

# Alterar numero de epocas de treinamento (~30-50 epocas para modelo bem treinado)
for epoch in range(10):
    mean_loss = 0.0
    model.train()
    # Iniciando uma epoch de treinamento supervisionado
    for batch in range(len(train_x)):
        x = train_x[batch].to(device)
        y = train_y[batch].to(device)
        m = mask[batch].to(device)
        optim.zero_grad()
        loss = -model(x, y, m)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optim.step()
        mean_loss += loss
    mean_loss /= len(train_x)
    mean_loss_history.append(mean_loss)
    

    model.eval()
    # Calculo do desempenho do modelo treinado nesta epoch
    y_pred = model.decode(valid_x[0].to(device), valid_mask[0].to(device))
    y_pred = IOBify(y_pred, idx2tag)
    y_true = IOBify(valid_y, idx2tag)
    f1 = f1_score(y_true, y_pred)
    f1_history.append(f1)

    print(f'Epoch: {epoch} | Loss media: {mean_loss} | f1-score: {f1}')



ValueError: Found input variables with inconsistent numbers of samples:
[3, 13, 4, 37, 39, 40, 26, 32, 36, 41, 43, 27, 36, 33, 30, 24, 11, 3, 9, 4, 16, 14, 15, 9, 32, 20, 37, 27, 27, 29, 28, 3, 8, 4, 8, 11, 15, 6, 10, 13, 10, 11, 12, 4, 5, 3, 7, 8, 14, 6, 6, 6, 11, 4, 12, 4, 14, 12, 4, 13, 3, 13, 14, 10, 11, 14, 10, 3, 13, 8, 6, 10, 10, 10, 3, 10, 13, 10, 14, 3, 10, 4, 19, 42, 22, 36, 28, 30, 22, 30, 20, 16, 3, 8, 4, 6, 7, 15, 4, 3, 11, 4, 19, 4, 15, 9, 5, 3, 12, 4, 40, 31, 22, 19, 31, 3, 14, 5, 28, 25, 39, 9, 9, 6, 12, 41, 18, 30, 33, 21, 3, 10, 5, 34, 17, 21, 20, 3, 11, 4, 7, 7, 10, 10, 12, 17, 12, 9, 9, 9, 9, 9, 9, 9, 9, 9, 3, 11, 4, 6, 9, 6, 6, 6, 6, 6, 6, 15, 11, 16, 7, 8, 9, 8, 9, 8, 9, 8, 9, 3, 12, 5, 23, 8, 16, 8, 22, 16, 18, 19, 18, 8, 22, 18, 8, 17, 18, 19, 47, 39, 44, 22, 3, 9, 13, 9, 15, 3, 13, 10, 9, 15, 4, 3, 45, 3, 12, 5, 5, 13, 11, 4, 4, 6, 8, 7, 8, 8, 8, 4, 7, 7, 8, 7, 8, 4, 7, 7, 8, 8, 7, 6, 5, 5, 5, 6, 5, 5, 4, 4, 6, 7, 7, 7, 8, 7, 4, 7, 9, 7, 7, 7, 4, 8, 8, 7, 8, 7,

In [ ]:
device = torch.device("cpu")
mean_loss_history = [y.to(device).detach().numpy() for y in mean_loss_history]
#tensor.detach().numpy()

In [ ]:
print(mean_loss_history[0])

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2)
axs[0].plot(f1_history)
axs[1].plot(mean_loss_history)
axs[0].set(ylabel='f1-score')
axs[1].set(xlabel='epochs', ylabel='mean loss')





# Utilização do modelo treinado para predição em novos dados e interface com usuário

Passos:


1.   Transformar as palavras do texto em indices utilizando o dicionario word2idx e converter para tensor
2.   Realizar as predições
3.   Realizar extração das entidades de acordo com as predições



In [ ]:
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

In [ ]:
#x_teste_path = '/content' + '/ner_train_sem_label_teste_01_ajuste.txt'
#x_teste_path = '/content' + '/ner_train_sem_label_teste_01.txt'
x_teste_path = '/content' + '/ner_train_sem_label_teste_1.txt'
f_x = open(x_teste_path, 'r').read().splitlines()
len(f_x)

In [ ]:
f_x_l = []
for i in f_x:
  #print(i)
  if i != '':
    f_x_l.append(i)
len(f_x_l)
#print(y_pred_1[1:20])
f_x = f_x_l

In [ ]:
len(f_x)

In [ ]:
from torch.utils.data import Dataset, DataLoader

x_test_dataloader = DataLoader(dataset = f_x, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

print(len(x_test_dataloader))

In [ ]:
#Transformar as palavras do texto em indices utilizando o dicionario word2idx e converter para tensor

for i, data_batch in enumerate(x_test_dataloader):
  if i % 10 == 0: print(i)
  text = f_x
  num_text = torch.LongTensor([word2idx['<UNK>'] if word not in word2idx else word2idx[word] for word in text]).unsqueeze(dim=0)
  mask = num_text != -1
  num_text.shape, mask.shape

### Passo 2: Realizar predições

In [53]:
#!nvcc --version
model = model.to('cpu')

In [54]:
y_pred_o = model.decode(num_text, mask)
y_pred_o = IOBify(y_pred_o, idx2tag)
#y_pred = IOBify(y_pred)
#for idx, (word, tag) in enumerate(zip(text, y_pred_o[0])):
#for idx, (word) in enumerate(zip(text, y_pred[0])):
#    print(word)
#    if idx >= 20:
#        break


NameError: name 'num_text' is not defined

In [ ]:
len(y_pred_o)
y_pred_o

### Passo 3: Realizar extração das entidades de acordo com as predições

In [ ]:
def extract_entities(y_pred, text):
    inside_entity = False
    entity_type = ''
    entity = ''
    entities = []
    for word, tag in zip(text, y_pred[0]):
        if not inside_entity:
            if tag[0] == 'B':
                inside_entity = True
                entity = word
                entity_type = tag[2:]
        else:
            if tag[0] == 'B':
                entities.append((entity, entity_type ))
                entity = word
                entity_type = tag[2:]
                inside_entity = True
            elif tag[0] == 'I':
                entity = entity + ' ' + word
                inside_entity = True
            else:
                entities.append((entity, entity_type))
                entity = ''
                entity_type = 'O'
                inside_entity = False
    return entities

entities = extract_entities(y_pred_o, text)

In [ ]:
entities

In [ ]:
valid_y_o_list = []
for i in valid_y_o:
   if not isinstance(i, list):
      valid_y_o_list.append(i)
   else:
      for j in i:
        valid_y_o_list.append(j)

len(valid_y_o_list)

In [ ]:
y_pred_o_list = []
for i in y_pred_o:
   if not isinstance(i, list):
      y_pred_o_list.append(i)
   else:
      for j in i:
        y_pred_o_list.append(j)

len(y_pred_o_list)

In [ ]:
from seqeval.metrics import f1_score
f1_score([valid_y_o_list], y_pred_o)

#len(y_pred)
#len(y_true_list)

In [ ]:
from seqeval.metrics import accuracy_score
accuracy_score([valid_y_o_list], y_pred_o)

In [ ]:
label = list(idx2tag.values())
label.remove('<PAD>')
label.remove('O')
label

In [ ]:
from seqeval.metrics import classification_report
# group B and I results

sorted_labels = sorted(
    label,
    key=lambda name: (name[1:], name[0])
)

print(classification_report([valid_y_o_list], [y_pred_o_list]))

In [ ]:
import sklearn.metrics
from sklearn import metrics
from sklearn_crfsuite import metrics
from sklearn.metrics import classification_report


sorted_labels = sorted(
    label,
    key=lambda name: (name[1:], name[0])
)
print(sklearn.metrics.classification_report(valid_y_o_list, y_pred_o_list,labels=['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC'],target_names=sorted_labels, digits=3))